In [4]:
import urllib3
import certifi
import re
import pandas as pd
from IPython.display import display, HTML
import pymongo
import json




In [46]:
#Construct all regular expressions
#title_regex = re.compile(r"class=\"title\"><h3>(.+)</h3>")
#date_regex = re.compile(r"class=\"clamp-details\">\s+<span>(.+)</span>")
#description_regex = re.compile(r"<div class=\"summary\">\s*([/S/s]+?)")  #incomplete
#score_regex = re.compile(r"")
#image_regex = re.compile(r"<a href=\"/movie/.*\"><img src=\"(.*)\" alt=\"")

title_formula = '><h3>(.*)<\/h'                        #Title
date_formula = 's">\s+<span>(.*)<'                    #Release Date  
description_formula = '>(.*)<\/div>\s+<\/a>\s+<\/div>\s+<s'  #Score
score_formula = 'e-wrap.*\n.*<img src="(.*)" a'        #Thumbnail URL
image_formula = 'y">\s+(.*)\s+<\/d'                    #Description


#title_formula = r"><h3>(\?'Name'.*)<\/h"
#date_formula = r"s\">\\s+<span>(\?'Release_Date'.*)<"
#description_formula = r"y\">\\s+(\?'Desc'.*)\\s+<\\/d"
#score_formula = r">(\?'CriticScore'.*)<\\/div>\\s+<\\/a>\\s+<\\/div>\\s+<s"
#image_formula = r"e-wrap.*\\n.*<img src=\"(\?'ThumbURL'.*)\" a"

title_regex = re.compile(title_formula)
date_regex = re.compile(date_formula)
description_regex = re.compile(description_formula)
score_regex = re.compile(score_formula)
image_regex = re.compile(image_formula)

#Construct an HTTP pool for connections
http = urllib3.PoolManager(ca_certs=certifi.where())


><h3>(.*)<\/h


In [27]:
#Get Connection string from JSON file
# Opening JSON file
json_file = open('C:\\Users\\stuar\\Downloads\\MongoDBConnection.json')
  
#Grab connection string
mongo_connection_string = json.load(json_file)['mongodb']

# Closing file
json_file.close() 


In [32]:
#Method from https://www.geeksforgeeks.org/mongodb-and-python/?ref=gcse

#Connect to db
client = pymongo.MongoClient(mongo_connection_string)

# Access database
da320_database = client['metacritic']
  
# Access collection of the database
metacritic_data = da320_database['movies']

metacritic = pd.DataFrame(metacritic_data.find())

In [51]:

def metacritic_scraper(year, page) -> pd.DataFrame:

    #Fetch the webpage
    url = f"https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected={year}&sort=dessc&view=detailed&page={page}"
    response = http.request('GET', url, headers={'User-Agent': 'Mozilla/5.0'})
    datastring = str(response.data, "utf-8")

    #Execute all the regular expressions
    titles = title_regex.findall(datastring)
    dates = date_regex.findall(datastring)
    descriptions = description_regex.findall(datastring)
    scores = score_regex.findall(datastring)
    images = image_regex.findall(datastring)

    #Return a unifed collection
    dataset = {"title": titles, "date": dates}#, "description": descriptions, "score": scores, "image": images}
    return pd.DataFrame(dataset)



In [53]:
import re 
import time

# Write a CSV file with this data 
for year in range(2000, 2023): 
    page = 0
    print(f"Collecting data for {year} page (page)...") 

    #Get first page
    data = metacritic_scraper(year, page) 

    #Continue until we reach a page with zero rows 
    while len(data) > 0: 
        #Convert the dataframe into a list of movies to insert into MongoDB 
        movies_to_insert = []
        for row in data.itertuples(): 
            movie = { "title": row.title, 
            "release_date": row.date,
            "description": row.description, 
            "metascore": row.score, 
            "image_url": row.image,
            }
	
            movies_to_insert.append(movie) 
        
        #Insert records into MongoDB
        print(f"Inserting {len(movies_to_insert)} movies for the year {year} page (pager) ")
        metacritic_data.insert_many(movies_to_insert) 
        page = page + 1 
        #Get next page
        data = metacritic_scraper(year, page) 


AttributeError: 'Pandas' object has no attribute 'descriptions'

**Connect to Metacritic and retrieve HTML**

In [50]:

display(HTML(data.to_html()))

,title,date
0,Girlhood,"October 29, 2003"
1,The Cooler,"November 26, 2003"
2,Horns and Halos,"February 28, 2003"
3,Zero Day,"August 3, 2003"
4,Friday Night,"June 30, 2003"
5,Kill Bill: Vol. 1,"October 10, 2003"
6,Monsieur Ibrahim,"December 5, 2003"
7,Stoked: The Rise and Fall of Gator,"August 22, 2003"
8,The Cuckoo,"July 11, 2003"
9,In My Skin,"November 7, 2003"


In [24]:

#Grab a connection from the connection pool
http = urllib3.PoolManager(ca_certs=certifi.where())

#Fixes bombing error in VS Code.  Thanks to Jamie Kirsila for the fix here.  This is not needed in Google Colab, for whatever reason.
http.headers={'User-Agent': 'Mozilla/5.0'}

#Retrieve the webpage
resp = http.request('GET', url)

#Grab the html of the webpage
datastring = str(resp.data, "utf-8")

Get connection string from file for connecting to MongoDB

In [25]:

#Method from https://www.geeksforgeeks.org/read-json-file-using-python/

# Opening JSON file
json_file = open('C:\\Users\\stuar\\Downloads\\MongoDBConnection.json')
  
# returns JSON object as 
# a dictionary
data = json.load(json_file)

# Grab connection info from JSON file
uri = data['mongodb']

# Closing file
json_file.close() 

Connect to MongoDB

In [41]:
#Method from https://www.geeksforgeeks.org/mongodb-and-python/?ref=gcse

#Connect to db
client = pymongo.MongoClient(uri)

# Access database
mydatabase = client['metacritic2']
  
# Access collection of the database
mycollection = mydatabase['movies']


In [27]:
#Make an array of the regular expressions
regex = ['','','','','']
regex[0] = '><h3>(.*)<\/h'                        #Title
regex[1] = 's">\s+<span>(.*)<'                    #Release Date  
regex[2] = '>(.*)<\/div>\s+<\/a>\s+<\/div>\s+<s'  #Score
regex[3] = 'e-wrap.*\n.*<img src="(.*)" a'        #Thumbnail URL
regex[4] = 'y">\s+(.*)\s+<\/d'                    #Description

In [28]:
#Make an array of the set of matches for each regex.  This includes extra data we don't want for this assignment, so this won't be our final stop.
#The line of code used here was adapted from the code generator on RegEx101
matches = []
for x in range(5):
    matches.append(re.finditer(regex[x], datastring, re.MULTILINE))


In [29]:
#Instantiate an array for each field
Title = []
ReleaseDate = []
CriticScore = []
ThumbnailURL = []
Description = []

In [30]:
#Using a Dataframe just to try to refresh my skills from my Python class.  It has a lot of useful features like easy sorting, if you need them.
df = pd.DataFrame()
df.insert(0,"Title", [])
df.insert(1,"Release Date", [])
df.insert(2,"Critic Score", [])
df.insert(3,"Thumbnail URL", [])
df.insert(4,"Description", [])

In [31]:
#Strip out just the text from the matches and put it in an array.  
#The line of code used here was adapted from the code generator on RegEx101
for matchNum, match in enumerate(matches[0], start=1):
    Title.append(match.group(1))
for matchNum, match in enumerate(matches[1], start=1):
    ReleaseDate.append(match.group(1))
for matchNum, match in enumerate(matches[2], start=1):
    CriticScore.append(match.group(1))
for matchNum, match in enumerate(matches[3], start=1):
    ThumbnailURL.append(match.group(1))
for matchNum, match in enumerate(matches[4], start=1):
    Description.append(match.group(1))

In [32]:
#Make each array a column in the dataframe
df = pd.DataFrame()
df.insert(0,"Title", Title)
df.insert(1,"Release Date", ReleaseDate)
df.insert(2,"Critic Score", CriticScore)
df.insert(3,"Thumbnail URL", ThumbnailURL)
df.insert(4,"Description", Description)

In [49]:
for index, row in df.iterrows():    
    mydatabase['movies'].insert_one(row.to_dict)
    print(row.to_dict)
    #movies_json = movie.to_json()
    #print(movie)

<bound method Series.to_dict of Title                                                      Boyhood
Release Date                                         July 11, 2014
Critic Score                                                   100
Thumbnail URL    https://static.metacritic.com/images/products/...
Description      Filmed over 12 years with the same cast, Richa...
Name: 0, dtype: object>
<bound method Series.to_dict of Title                                                   Mr. Turner
Release Date                                     December 19, 2014
Critic Score                                                    94
Thumbnail URL    https://static.metacritic.com/images/products/...
Description      Mr. Turner explores the last quarter century o...
Name: 1, dtype: object>
<bound method Series.to_dict of Title                                                    Leviathan
Release Date                                     December 25, 2014
Critic Score                                        